In [149]:
import pickle
import pandas as pd

def get_class_name(filename):
    if 'cat' in filename:
        class_name = 0
    else:
        class_name = 1 
    return class_name

def load_data(dataset='Train'):    
    return pd.read_pickle('../data_processed/' + dataset + '_MELandDeltas.pkl')

def get_dimensions(shape='mel_only', frames=None):
    if shape=='mel_only':            
        mel_height = 128
        mel_depth = 1
    elif shape=='mel_delta':
        mel_height = 256
        mel_depth = 1        
    elif shape=='mel_delta_stacked':
        mel_height = 128
        mel_depth = 2        
    mel_width = int(frames.shape[0]/mel_height/mel_depth)
    return mel_height, mel_width, mel_depth    

In [207]:
def extract_mel_spectrograms(dataset='Train', features=['Mel'], shape='mel_only'):
    
    df = load_data(dataset)
        
    #Where it will be stored
    files = []
    labels = []
    data = []
    
    #List of file names in the dataset
    file_names = list(df.File_id.unique())
    
    for file in file_names:
        
        class_name = get_class_name(file)        
        #Filter for the file and extract needed features        
        frames = np.array(df[df['File_id'] == file][features])
        frames = frames.ravel()
        frames = np.concatenate(frames)
        
        #obtain some dimentions about the set to load
        mel_height, mel_width, mel_depth = get_dimensions(shape=shape, frames=frames)

        #Combine all the frames into a mel_spectrogram
        try:
            mel = np.reshape(frames, (mel_height, mel_width, mel_depth))
        except ValueError:                      
            print(file)
            mel = np.reshape(frames, (mel_height, mel_width, mel_depth))
            
        
        #each mel needs to be chopped into segments of 28 width
        batch_size = int(mel.shape[1] / 28)
        for i in list(range(batch_size)):
            labels.append(class_name)
            files.append(file)            
            data.append(mel[:,i*28:(i+1)*28])
            
    return np.array(data, dtype=np.float32), np.array(labels), np.array(files)

In [214]:
x_train, y_train, f_train = extract_mel_spectrograms(dataset='Train', features=['Mel', 'Mel_deltas'], 
                                                    shape='mel_delta_stacked')